<center><h1>Data Wrangling Step: Diabetes Data Set</h1></center>

This notebook shows the steps our group took to clean and preprocess our data set for our group project in IST687.  We got this data set from [UCI Machine Learning Datasets Repository--Diabetes 130-US hospitals for years 1999-2008 Data Set](https://archive.ics.uci.edu/ml/datasets/diabetes+130-us+hospitals+for+years+1999-2008).  Our goal is to use the data provided to build a classifier that can predict whether someone admitted for diabetes will be readmitted in <30 days, >30days, or not readmitted.  

This notebook details the steps taken to clean the data set so that machine learning alogrithms can be applied to it. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
pd.options.display.max_rows = 4000

<center><h2>Data Dictionary: Categorical Encodings</h2></center>

**admission\_type\_id **|**description**
:-----:|:-----:
1 |Emergency
2 |Urgent
3 |Elective
4 |Newborn
5 |Not Available
6 |NULL
7 |Trauma Center
8 |Not Mapped
 | 
**discharge\_disposition\_id** |**description**
1 |Discharged to home
2 |Discharged/transferred to another short term hospital
3 |Discharged/transferred to SNF
4 |Discharged/transferred to ICF
5 |Discharged/transferred to another type of inpatient care institution
6 |Discharged/transferred to home with home health service
7 |Left AMA
8 |Discharged/transferred to home under care of Home IV provider
9 |Admitted as an inpatient to this hospital
10 |Neonate discharged to another hospital for neonatal aftercare
11 |Expired
12 |Still patient or expected to return for outpatient services
13 |Hospice / home
14 |Hospice / medical facility
15 |Discharged/transferred within this institution to Medicare approved swing bed
16 |Discharged/transferred/referred another institution for outpatient services
17 |Discharged/transferred/referred to this institution for outpatient services
18 |NULL
19 |Expired at home. Medicaid only, hospice.
20 |Expired in a medical facility. Medicaid only, hospice.
21 |Expired, place unknown. Medicaid only, hospice.
22 |Discharged/transferred to another rehab fac including rehab units of a hospital .
23 |Discharged/transferred to a long term care hospital.
24 |Discharged/transferred to a nursing facility certified under Medicaid but not certified under Medicare.
25 |Not Mapped
26 |Unknown/Invalid
30 |Discharged/transferred to another Type of Health Care Institution not Defined Elsewhere
27 |Discharged/transferred to a federal health care facility.
28 |Discharged/transferred/referred to a psychiatric hospital of psychiatric distinct part unit of a hospital
29 |Discharged/transferred to a Critical Access Hospital (CAH).
 | 
**admission\_source\_id** |**description**
1 |Physician Referral
2 |Clinic Referral
3 |HMO Referral
4 |Transfer from a hospital
5 |Transfer from a Skilled Nursing Facility (SNF)
6 |Transfer from another health care facility
7 |Emergency Room
8 |Court/Law Enforcement
9 |Not Available
10 |Transfer from critial access hospital
11 |Normal Delivery
12 |Premature Delivery
13 |Sick Baby
14 |Extramural Birth
15 |Not Available
17 |NULL
18 |Transfer From Another Home Health Agency
19 |Readmission to Same Home Health Agency
20 |Not Mapped
21 |Unknown/Invalid
22 |Transfer from hospital inpt/same fac reslt in a sep claim
23 |Born inside this hospital
24 |Born outside this hospital
25 |Transfer from Ambulatory Surgery Center
26 |Transfer from Hospice
<br>
<br>


<center><h1>Steps Taken During Data Cleaning</h1></center>

-- (x) Convert `gender` values from strings to integers.  Male=0, Female=1.

-- (x) Convert `diabetesMed` values from strings to integers.  No=0, Yes=1

-- (x) Get list of column names for all categorical columns.  All medicine columns had the same possible values of `Down`, `No`, `Steady`, or `Up`.  Iterate through all columns and get unique values as a list. Sort the list.  If the list is equal to `['Down', 'No', 'Steady', 'Up']`, this is a medicine column, and needs to be one hot encoded.  Append column name to `one_hot_cols` array.  Also add `race` to this column.  When finished, create new one-hot encoded dataframe using `pd.get_dummies()` on the dataframe and list of column names.  

-- (X) Drop encounterID.  

-- (x) Change age category from current format to integer value.  E.G. Row 1, \[10-20) becomes 15.  

-- Deal with missing weight values 

-- Create correlation heatmap with labels

-- Create Correlation heatmap between variables

-- Decide if we should drop `patient_nbr` column.  If not, figure out how we are going to encode it, since the current format is a problem.  



<center><h3>Step 1: Read in the data</h3></center>

In this step, we read in the raw dataset, store it in a pandas dataframe, and store the labels as a separate series before dropping the column from the dataframe.  The labels were removed to make the data conversion steps easier, since we want to leave the columns encoded as strings for now.  

In [2]:
# Read in data and remove labels for now.  Will add back in when dealing with null values.  

raw_df = pd.read_csv("diabetic_data.csv")

# Remove labels and store in a separate variable. Will add back in after one hot encoding step
labels = raw_df['readmitted']
raw_df.drop('readmitted', axis=1, inplace=True)

<center><h3>Step 2:  Recode Binary Categorical Columns</h3></center>

The columns `gender`, `diabetesMed`, and `change` are categorical columns that contain only choices. These columns are currently encoded as strings.  Here, we replace the categorical values with 0s and 1s.  The mapping for the new encodings for each column is contained in the corresponding comment above each line of code.  

In [3]:
# Recode gender column, Female = 0, Male = 1
raw_df["gender"] = (raw_df["gender"].values == "Female" ).astype(np.uint8)

# Recode diabetesMed column, No = 0, Yes = 1
raw_df['diabetesMed'] = (raw_df["diabetesMed"].values == "Yes").astype(np.uint8)

# Recode change column, No = 0, Ch = 1
raw_df["change"] = (raw_df['change'].values == "Ch").astype(np.uint8)

<center><h3>Step 3: One-Hot Encoding Categorical Columns</h3></center>

The strong majority of the columns in this data set are multicategoircal, and encoded as strings.  Some of these, such as `race`, were easy to identify.  However, because the of sheer amount of medicine columns, we scripted this step to make things easier.  We started by identifying that all columns 

In [4]:
# Search for all medicine columns by checking the column's unique types.  Append any category name that fits

categorical_cols = ['race', 'A1Cresult']
for col in raw_df.columns.values:
    target = ['Down', 'No', 'Steady', 'Up']
    actual = list(sorted(raw_df[col].unique()))
    if target == actual:
        categorical_cols.append(col)

# One hot encode the categorical columns using pd.get_dummies()
other_cat_cols = ['max_glu_serum', 'acetohexamide', 'tolbutamide', 'troglitazone', 
                 'tolazamide', 'examide', 'citoglipton', 'glipizide-metformin', 
                 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone']
categorical_cols.extend(other_cat_cols)
one_hot_df = pd.get_dummies(raw_df, columns=categorical_cols)
age_str_vector = list(sorted(one_hot_df['age']))
age_str_vector = age_str_vector[::-1]

<center><h3>Step 4: Convert Age column to integer values</h3></center>

Age values are current stored as strings denoting binned ages. For instance, a 37 year old patient would show as "\[30-40)".  Here, we replace all values with the median value for each bin--in this example case, patient's age would change to the integer 35.

In [5]:
# Create a vector of integers that contain the average of the age bounds for each row.  
age_int_vector = []
for row in age_str_vector:
    age = int(row[1] + '5')
    age_int_vector.append(age)

# Replace the age column with the new list containing corresponding integer values    
one_hot_df['age'] = age_int_vector

<center><h3>Step 4: Drop Unneccesary Columns</h3></center>

Here, we remove columns that contain information that is not useful, because of the format of the data, or because of excessive missing values.  

In [6]:
# Drop encounter_id column

one_hot_df.drop(['admission_type_id', 'discharge_disposition_id', 'admission_source_id', 'encounter_id', 'patient_nbr', 'payer_code', 'medical_specialty', 'weight'], axis=1, inplace=True)

<center><h3>Step 5: Strip letters from Diag Columns and cast to Floats</h3></center>

Most values in these columns `Diag_1`, `Diag_2`, and `Diag_3` are able to be cast to floats without any changes, but some values are missing, and others contain the letter 'E' or 'V' which need to be removed before the data can be cast to a numeric type.  

To deal with the null values, this required calculating the median of the column.  However, this cannot be done correctly until all other values are stripped of letters and cast to floats.  To accomplish this, we wrote a helper function that kept track of the indexes of null values dealing with all cases except for the null values.  Then, we calculated the median and replaced all the null values with this value.  

In [7]:
diags = ['diag_1', 'diag_2', 'diag_3']

def update_diags(column):
    column = list(column)
    cleaned_column = []
    all_values = []
    nan_indexes = []
    # Iterate through list
    for ind, i in enumerate(column):
        if i[0] == 'E' or i[0] == 'V':
            cleaned_column.append(float(i[1:]))
            all_values.append(float(i[1:]))
        elif i == '?' or 'a' in i:
            nan_indexes.append(ind)
            cleaned_column.append(i)
        else:
            cleaned_column.append(float(i))
            all_values.append(float(i))
            
    #get median value
    median_val = np.median(all_values)
    # Replace all nan values with median values
    for n in nan_indexes:
        cleaned_column[n] = median_val
    # Cast column to a pandas Series and return it
    return pd.Series(cleaned_column)
    
            
for column in diags:
    one_hot_df[column] = update_diags(one_hot_df[column])
    print(one_hot_df[column].isna().any())
    print(one_hot_df[column].dtype)

False
float64
False
float64
False
float64


<center><h3>Final Step: Add Labels Back in and Export Dataset</h3></center>

For the final step, we confirmed that there are no missing values left in the data set.  We then added the labels back in, and exported the newly cleaned and transformed data set as a .csv file.  

In [8]:
# Check that no question mark values remain in the data set
# print(one_hot_df.apply(lambda x: '?' in x.values, axis=1).any())

# Check that no NaN Values remain in the dataset
# one_hot_df.isna().any().any()

In [9]:
# Add labels column back into dataframe and write to csv
one_hot_df['labels'] = labels
one_hot_df.to_csv("cleaned_diabetes_data_v2")